In [281]:
import numpy as np
import qiskit
from qiskit.quantum_info import state_fidelity
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy
import h5py

import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import construct_ising_local_term, reduce_list, X, I2, get_perms
from ansatz_sparse import ansatz_sparse
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector
from scipy.linalg import expm
from itertools import product
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

J, h, g = (1, 0, 3)
def eval_energy(config):
    e = 0
    for perm in perms_1+perms_2+perms_3:
        for j in range(len(perm)//2):
            e += 1 if config[perm[2*j]]==config[perm[2*j+1]] else -1
    return e
e = 48
gs = '0'*L
gss = []
bitstrings = [''.join(bits) for bits in product('01', repeat=16)]
for bitstring in bitstrings:
    if eval_energy(bitstring) < e:
        gs = bitstring
        e = eval_energy(bitstring)
    if eval_energy(bitstring) == -16:
        gss.append(bitstring)
ground_states = gss
n = len(ground_states[0])
dim = 2**n
state = np.zeros(dim, dtype=complex)
for s in ground_states:
    index = int(s, 2)
    state[index] = 1 / np.sqrt(len(ground_states))
psi = Statevector(state)

Lx, Ly = (4, 4)
L = Lx*Ly
t = 0.125
# construct Hamiltonian
latt = qib.lattice.TriangularLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()

perms_1 = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 0, 5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12]]
perms_2 = [[0, 5, 10, 15, 3, 4, 9, 14, 2, 7, 8, 13, 1, 6, 11, 12], [5, 10, 15, 0, 4, 9, 14, 3, 7, 8, 13, 2, 6, 11, 12, 1]]
perms_3 = [[0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15], [4, 8, 12, 0, 5, 9, 13, 1, 6, 10, 14, 2, 7, 11, 15, 3]]

#eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=100)
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=10, v0=psi.data, which='SA')
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]

eigenvalues_sort[0]

-51.085742197520226

In [282]:
Lx, Ly = (4, 4)
L = Lx*Ly
perms_1 = [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [1, 2, 3, 0, 5, 6, 7, 4, 9, 10, 11, 8, 13, 14, 15, 12]]
perms_2 = [[0, 5, 10, 15, 3, 4, 9, 14, 2, 7, 8, 13, 1, 6, 11, 12], [5, 10, 15, 0, 4, 9, 14, 3, 7, 8, 13, 2, 6, 11, 12, 1]]
perms_3 = [[0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15], [4, 8, 12, 0, 5, 9, 13, 1, 6, 10, 14, 2, 7, 11, 15, 3]]

In [230]:
"""Lx, Ly = (6, 6)
L = Lx*Ly
perms_1 = [[i for i in range(36)], 
           [1, 2, 3, 4, 5, 0,
            7, 8, 9, 10, 11, 6,
           13, 14, 15, 16, 17, 12, 
           19, 20, 21, 22, 23, 18,
           25, 26, 27, 28, 29, 24,
           31, 32, 33, 34, 35, 30]]
perms_2 = [[0, 7, 14, 21, 28, 35, 
            1, 8, 15, 22, 29, 30, 
            2, 9, 16, 23, 24, 31, 
            3, 10, 17, 18, 25, 32, 
            4, 11, 12, 19, 26, 33, 
            5, 6, 13, 20, 27, 34], 
           [7, 14, 21, 28, 35, 0,
           8, 15, 22, 29, 30, 1,
           9, 16, 23, 24, 31, 2,
           10, 17, 18, 25, 32, 3,
           11, 12, 19, 26, 33, 4, 
           6, 13, 20, 27, 34, 5]]
perms_3 = [[0, 6, 12, 18, 24, 30,
            1, 7, 13, 19, 25, 31, 
            2, 8, 14, 20, 26, 32,
            3, 9, 15, 21, 27, 33, 
            4, 10, 16, 22, 28, 34, 
            5, 11, 17, 23, 29, 35], 
           [6, 12, 18, 24, 30, 0,
           7, 13, 19, 25, 31, 1,
           8, 14, 20, 26, 32, 2,
           9, 15, 21, 27, 33, 3, 
           10, 16, 22, 28, 34, 4, 
           11, 17, 23, 29, 35, 5]]"""

'Lx, Ly = (6, 6)\nL = Lx*Ly\nperms_1 = [[i for i in range(36)], \n           [1, 2, 3, 4, 5, 0,\n            7, 8, 9, 10, 11, 6,\n           13, 14, 15, 16, 17, 12, \n           19, 20, 21, 22, 23, 18,\n           25, 26, 27, 28, 29, 24,\n           31, 32, 33, 34, 35, 30]]\nperms_2 = [[0, 7, 14, 21, 28, 35, \n            1, 8, 15, 22, 29, 30, \n            2, 9, 16, 23, 24, 31, \n            3, 10, 17, 18, 25, 32, \n            4, 11, 12, 19, 26, 33, \n            5, 6, 13, 20, 27, 34], \n           [7, 14, 21, 28, 35, 0,\n           8, 15, 22, 29, 30, 1,\n           9, 16, 23, 24, 31, 2,\n           10, 17, 18, 25, 32, 3,\n           11, 12, 19, 26, 33, 4, \n           6, 13, 20, 27, 34, 5]]\nperms_3 = [[0, 6, 12, 18, 24, 30,\n            1, 7, 13, 19, 25, 31, \n            2, 8, 14, 20, 26, 32,\n            3, 9, 15, 21, 27, 33, \n            4, 10, 16, 22, 28, 34, \n            5, 11, 17, 23, 29, 35], \n           [6, 12, 18, 24, 30, 0,\n           7, 13, 19, 25, 31, 1,\n          

In [283]:

Vlists = {}
for t in [0.125]:
    #with h5py.File(f"./results/triangularTFIM_ccU_SPARSE_10{g}_Lx4Ly4_t{t}_layers9_niter5_rS1_2hloc.hdf5", "r") as f:
    #    Vlists[t] = f["Vlist"][:]
    with h5py.File(f'./results/triangularTFIM_ccU_SPARSE_10{g}_Lx4Ly4_t{t}_layers9_niter10_rS1_2hloc.hdf5') as f:
        Vlists[t]  =  f["Vlist"][:]
perms_extended = [[perms_1[0]]] + [perms_1] + [[perms_1[0]], [perms_2[0]]] +\
                    [perms_2] + [[perms_2[0]], [perms_3[0]]] + [perms_3] + [[perms_3[0]]] 
perms_ext_reduced = [perms_1]  + [perms_2] + [perms_3]
control_layers = [0, 2, 3, 5, 6, 8]

In [284]:
"""
    Test operations.
"""

for t in [0.125]:
    Vlist = Vlists[t]
    Vlist_reduced = []
    for i, V in enumerate(Vlist):
        if i not in control_layers:
            Vlist_reduced.append(V)
    
    state = random_statevector(2**L).data
    print("Trotter error of the optimized point: ", (np.linalg.norm(ansatz_sparse(Vlist, L, perms_extended, state) - expm_multiply(
        1j * t * hamil, state), ord=2) + np.linalg.norm(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state) - expm_multiply(
        -1j * t * hamil, state), ord=2))/2)
    
    print('infidelity: ', 1-(state_fidelity(ansatz_sparse(Vlist, L, perms_extended, state), expm_multiply(
        1j * t * hamil, state)) + state_fidelity(ansatz_sparse(Vlist_reduced, L, perms_ext_reduced, state), expm_multiply(
        -1j * t * hamil, state)))/2)

Trotter error of the optimized point:  0.06860187019219097
infidelity:  0.005051027399576968


In [241]:
sys.path.append("../../src/controlled_unitary_optimizer")
sys.path.append("../../src/brickwall_ansatz")
from optimize_3q import optimize_3q 
from utils_3q import make_controlled, random_unitary

Xlists_opts = {}
for t in [0.1, 0.11, 0.12]:
    Vlist = Vlists[t]
    Xlists_opt = {}
    perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2]]
    #perms_qc = [[0, 1], [0, 2]]
    
    for i in control_layers:
        cU = make_controlled(Vlist[i])
        f_best, err_best, Glist_best = (0, 2, None)
        for _ in range(10):
            Xlist_start = [random_unitary(4) for i in range(len(perms_qc))]
            Xlist, f_iter, err_iter = optimize_3q(L, cU, Xlist_start, perms_qc, niter=1000)
            if err_iter[-1] < err_best:
                f_best, err_best, Xlist_best = (f_iter[-1], err_iter[-1], Xlist)
        print("Best f: ", f_best)
        print("Best err: ", err_best)
        Xlists_opt[i] = Xlist_best
    Xlists_opts[t] = Xlists_opt

Best f:  -7.999999999999884
Best err:  2.4365152492788765e-07
Best f:  -7.9999999999998765
Best err:  2.582856070544144e-07
Best f:  -7.999999999999883
Best err:  2.6036693229531357e-07
Best f:  -7.999999999999858
Best err:  2.7234763755179536e-07
Best f:  -7.9999999999999485
Best err:  2.034530300312693e-07
Best f:  -7.999999999999903
Best err:  2.1808460059592356e-07
Best f:  -7.999999999999885
Best err:  2.433977709451925e-07
Best f:  -7.99999999999992
Best err:  1.9067862920042635e-07
Best f:  -7.999999999999862
Best err:  2.733987973992777e-07
Best f:  -7.999999999999933
Best err:  1.7658822326186053e-07
Best f:  -7.999999999999998
Best err:  5.0296928870394024e-08
Best f:  -7.999999999999911
Best err:  1.9404666189816711e-07
Best f:  -7.999999999999908
Best err:  2.337107459429008e-07
Best f:  -7.9999999999999645
Best err:  1.2902573366106161e-07
Best f:  -7.999999999999901
Best err:  2.166489886179899e-07
Best f:  -7.999999999999925
Best err:  2.215055311049115e-07
Best f:  -7.9

In [243]:
# Save the optimal gates.


for t in [0.1, 0.11, 0.12]:
    Xlists_opt = Xlists_opts[t]
    for i in control_layers:
        with h5py.File(f"./results/triangularTFIM_ccU_SPARSE_10{g}_Lx4Ly4_t{t}_layers25_niter5_rS1_DECOMPOSE_n{len(perms_qc)}_layer{i}.hdf5", "w") as file:
            file.create_dataset(f"Xlist_{i}", data=Xlists_opt[i])


In [101]:
#perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2]]
perms_qc = [[0, 1], [0, 2]]
Xlists_opt = {}
for i in control_layers:
    with h5py.File(f"./results/triangularTFIM_ccU_SPARSE_10{g}_Lx4Ly4_t{t}_layers25_niter5_rS1_DECOMPOSE_n{len(perms_qc)}_layer{i}.hdf5", "r") as file:
        Xlists_opt[i] = file[f"Xlist_{i}"][:]

In [286]:
from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp

for t in [0.125]:
    Vlist = Vlists[t]
    qc = qiskit.QuantumCircuit(L+1)
    qc.x(L)
    for i, V in enumerate(Vlist):
        layer = i
        #Xlists_opt = Xlists_opts[t]
        if i in control_layers:
            """Glist = Xlists_opt[i]
            qc_3 = qiskit.QuantumCircuit(3)
            for j, G in enumerate(Glist):
                qc_3.unitary( G, (3-1-perms_qc[j][1], 3-1-perms_qc[j][0]))
    
            for perm in perms_extended[layer]:
                for j in range(L//2):
                    qc.append(qc_3.to_gate(), [L-perm[2*j]-1, L-perm[2*j+1]-1, L])"""
            
            if i in [2, 5, 6]:
                continue
            elif i in [0, 8]:
                for perm in perms_extended[layer]:
                    for j in range(L//2):
                        qc.cy(L, L-1-perm[2*j])
                        qc.cz(L, L-1-perm[2*j+1])
            elif i==3:
                for perm in perms_extended[layer]:
                    for _ in range(L//Lx):
                        for j in range(Ly//2):
                            qc.cx(L, L-1-Lx*_-(2*j + (1 if _%2==0 else 0)))
            
        else:
            for perm in perms_extended[layer]:
                for j in range(L//2):
                    qc.unitary(V, [L-perm[2*j]-1, L-perm[2*j+1]-1])
    qc.x(L)


    t_ = t
    state = random_statevector(2**L).data
    qc_ext1 = qiskit.QuantumCircuit(L+1)
    qc_ext1.initialize(state, [i for i in range(L)])
    qc_ext1.append(qc.to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv1 = execute(transpile(qc_ext1), backend).result().get_statevector().data

    qc_ext2 = qiskit.QuantumCircuit(L+1)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    qc_ext2.append(qc.to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv2 = execute(transpile(qc_ext2), backend).result().get_statevector().data

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    exact_v1 = np.kron(ket_0, expm_multiply(1j * t_ * hamil, state))
    exact_v2 = np.kron(ket_1, expm_multiply(-1j * t_ * hamil, state))
    
    #err = (np.linalg.norm(sv1-exact_v1, ord=2) + np.linalg.norm(sv2-exact_v2, ord=2))/2
    err = 1-(state_fidelity(sv1, exact_v1)+state_fidelity(sv2, exact_v2))/2
    print(state_fidelity(sv2, exact_v2))
    print(state_fidelity(sv1, exact_v1))
    
    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext1, optimization_level=2, basis_gates=['initialize', 'unitary', 'cx', 'u3']), )
    count_ops = dag.count_ops()
    print(count_ops)

    ccU_errs_0125.append(err)
    ccU_cxs_0125.append(count_ops['unitary'])
    #print(f"t={t}, Gate Count: ", count_ops['unitary'] + count_ops['cx'], " State infidelity: ", err)
    print("Infidelity: ", err)

0.9923808610216577
0.9916783642031411
{'initialize': 1, 'u3': 66, 'cx': 40, 'unitary': 48}
Infidelity:  0.007970387387600586


In [289]:
"""
    Now here is to compare the performance of the ccU circuit
    with the 1st and 2nd order Trotter circuits, in terms of 
    gate count vs Trotter error. I demonstrate it on L=10 system.
"""

from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp


def controlled_trotter(c1, L, Lx, Ly, J, g, perms_1, perms_2, perms3, dag=False, dt=t/2):
    t = c1/2
    nsteps = int(t/dt)
    t = t/nsteps
    indices = [0, 1]
    coeffs = [1, 1]
    #indices = oc.SplittingMethod.suzuki(2, 1).indices
    #coeffs = oc.SplittingMethod.suzuki(2, 1).coeffs
    perms_ext = [perms_1, perms_2, perms_3]*len(indices)
    
    hloc1 = g*(np.kron(X, I2)+np.kron(I2, X))/6
    hloc2 = construct_ising_local_term(J, 0, 0, ndim=2)
    hlocs = (hloc1, hloc2)

    K = []
    for i, perms in enumerate(perms_ext):
        perm = perms[0]
        K_layer = [None for _ in range(L)]
        for j in range(len(perm)//2):
            K_layer[perm[2*j]] = CYGate
            K_layer[perm[2*j+1]] = CZGate
        K.append(K_layer)

    Vlist_start = []
    for i, c in zip(indices, coeffs):
        Vlist_start.append(scipy.linalg.expm(-1j*c*t*hlocs[i]))
    Vlist_gates = []
    for V in Vlist_start:
        qc2 = qiskit.QuantumCircuit(2)
        qc2.unitary(V, [0, 1], label='str')
        Vlist_gates.append(qc2)
    print(nsteps)

    qc = qiskit.QuantumCircuit(L+1)
    for n in range(nsteps):
        for layer, qc_gate in enumerate(Vlist_gates):

            perms = perms_1
            qc.x(L)
            for j in range(L):
                if K[3*layer][j]:
                    qc.append(K[3*layer][j](), [L, L-1-j])
            qc.x(L)
            for perm in perms:
                for j in range(len(perm)//2):
                    qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])
            qc.x(L)
            for j in range(L):
                if K[3*layer][j]:
                    qc.append(K[3*layer][j](), [L, L-1-j])
            qc.x(L)

            perms = perms_2
            qc.x(L)
            for j in range(L):
                if K[3*layer+1][j]:
                    qc.append(K[3*layer+1][j](), [L, L-1-j])
            qc.x(L)
            for perm in perms:
                for j in range(len(perm)//2):
                    qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])

            qc.x(L)
            for j in range(L):
                if K[3*layer+1][j]:
                    qc.append(K[3*layer+1][j](), [L, L-1-j])
            qc.x(L)

            perms = perms_3
            qc.x(L)
            for j in range(L):
                if K[3*layer+2][j]:
                    qc.append(K[3*layer+2][j](), [L, L-1-j])
            qc.x(L)
            for perm in perms:
                for j in range(len(perm)//2):
                    qc.append(qc_gate.to_gate(), [L-(perm[2*j]+1), L-(perm[2*j+1]+1)])
            qc.x(L)
            for j in range(L):
                if K[3*layer+2][j]:
                    qc.append(K[3*layer+2][j](), [L, L-1-j])
            qc.x(L)
    return qc

trotter1_cxs_0125_r2 = []
trotter1_errs_0125_r2 = []
for t_ in [t]:
    state = random_statevector(2**L).data
    qc_ext1 = qiskit.QuantumCircuit(L+1)
    qc_ext1.initialize(state, [i for i in range(L)])
    qc_ext1.append(controlled_trotter(2*t_, L, Lx, Ly, J, g, perms_1, perms_2, perms_3).to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv1_T = execute(transpile(qc_ext1), backend).result().get_statevector().data
    
    qc_ext2 = qiskit.QuantumCircuit(L+1)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    qc_ext2.append(controlled_trotter(2*t_, L, Lx, Ly, J, g, perms_1, perms_2, perms_3).to_gate(), [i for i in range(L+1)])
    backend = Aer.get_backend("statevector_simulator")
    sv2_T = execute(transpile(qc_ext2), backend).result().get_statevector().data

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    exact_v1 = np.kron(ket_0, expm_multiply(1j * t_ * hamil, state))
    exact_v2 = np.kron(ket_1, expm_multiply(-1j * t_ * hamil, state))
    #err = (np.linalg.norm(sv1_T-exact_v1, ord=2) + np.linalg.norm(sv2_T-exact_v2, ord=2))/2
    err = 1-(state_fidelity(sv1_T, exact_v1) + state_fidelity(sv2_T, exact_v2))/2
    
    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext1, optimization_level=2, basis_gates=['initialize', 'cx', 'unitary', 'u3']))
    count_ops = dag.count_ops()
    print(count_ops)

    trotter1_cxs_0125_r2.append(count_ops['cx']+count_ops['unitary'])
    trotter1_errs_0125_r2.append(err)

    print(f"t={t}, Gate count: ", count_ops['cx'])
    print("Infidelity: ", err)

2
2
{'initialize': 1, 'u3': 309, 'cx': 144, 'unitary': 192}
t=0.125, Gate count:  144
Infidelity:  0.040493270579260554


In [288]:
144+192

336

In [209]:
print(ccU_cxs_0125)
print(ccU_errs_0125)

[88, 160, 304, 592]
[0.007967802541555491, 0.01911296380278027, 0.03210115324981, 0.06857895447807816]


In [210]:
print(trotter2_cxs_0125_r1)
print(trotter2_errs_0125_r1)

[256, 496, 976, 1936]
[0.00782247971958605, 0.011745827245382534, 0.01890681727438681, 0.05414003585959826]


In [212]:
print(trotter1_cxs_0125_r2)
print(trotter1_errs_0125_r2)

[336, 656, 1296, 2576]
[0.04063820201992541, 0.07368754490680696, 0.05473561891297707, 0.06132877271376225]


In [ ]:
trTFIM_ccU_cxs_0125=[88, 160, 304, 592]
trTFIM_ccU_errs_0125=[0.007967802541555491, 0.01911296380278027, 0.03210115324981, 0.06857895447807816]
trTFIM_trotter2_cxs_0125_r1=[256, 496, 976, 1936]
trTFIM_trotter2_errs_0125_r1=[0.00782247971958605, 0.011745827245382534, 0.01890681727438681, 0.05414003585959826]
trTFIM_trotter1_cxs_0125_r2=[336, 656, 1296, 2576]
trTFIM_trotter1_cxs_0125_r2=[0.04063820201992541, 0.07368754490680696, 0.05473561891297707, 0.06132877271376225]